In [1]:
from indra_gpt.check_correctness import get_create_training_set, LOCAL_FILES, run_stats

curations_file = LOCAL_FILES.joinpath("indra_assembly_curations.json")
corpus = LOCAL_FILES.joinpath("indra_benchmark_corpus.json")

# The creation of the dataframe from the curations and statements takes ~60 min, mostly due to pre-caching of synonyms and definitions.
# If the output file already is cached, it will load from the cache instead.
training_data_df = get_create_training_set(curations_file=curations_file, statement_json_file=corpus)


INFO: [2023-03-27 15:56:08] check_correctness - agent_json_list dtype is str, using eval to convert to list of OrderedDicts


In [2]:
# Run the stats (takes ~1 h for 100 iterations)
results_dict = run_stats(training_data_df, n_iter=150)

Running chat completions: 100%|██████████| 150/150 [1:37:37<00:00, 39.05s/it]
INFO: [2023-03-27 17:33:48] check_correctness - Saving results...


Confusion matrix:
               correct  incorrect
gpt_correct         62         12
gpt_incorrect       31         45
Precision: 0.8378378378378378
Recall: 0.6666666666666666
Accuracy: 0.7133333333333334
Total examples: 150


In [19]:
import pandas as pd

# Get a dataframe of the results
results_df = pd.DataFrame(results_dict["chat_qa"])
results_df["correctly_labeled"] = ((results_df["response"] == "yes") & (results_df["tag"] == "correct")) | ((results_df["response"] == "no") & (results_df["tag"] != "correct"))

# Get the accuracy rate per tag as counted by 'correctly_labeled', how many rows has that tag and print the results
for tag in results_df["tag"].unique():
    print(f"{tag}: {results_df[results_df['tag'] == tag]['correctly_labeled'].mean():.2f} ({results_df[results_df['tag'] == tag].shape[0]})")

no_relation: 0.88 (24)
correct: 0.67 (93)
polarity: 0.86 (7)
grounding: 1.00 (4)
other: 0.00 (3)
act_vs_amt: 0.80 (5)
entity_boundaries: 0.50 (4)
wrong_relation: 1.00 (4)
agent_conditions: 1.00 (1)
mod_site: 0.00 (1)
hypothesis: 0.00 (1)
negative_result: 1.00 (3)
